In [2]:
import pymrio as mr
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
# from pypardiso import spsolve # package to perform fast matrix inversions 
from IPython.display import IFrame

In [3]:
FIGARO_path = '../FIGARO/'

Some documentation first. The methodological report for FIGARO provides all the elements necessary to rebuild easily the IO system.
![FIGARO structure](FIGARO_pxp_structure.png "FIGARO structure")

We start with importing the raw data for the product-to-product IO tables, available at https://ec.europa.eu/eurostat/documents/51957/12789261/IOP.7z/cc10fa4e-7f6a-21a8-64f1-10b089761568?t=1621882849396

In [4]:
T = pd.read_csv(FIGARO_path + 'ICIOP_64_30_2017.csv',
               header=[0,1],
               index_col=[0,1])

In [5]:
T

BE                                                    \
                 CPA_A01  CPA_A02  CPA_A03     CPA_B CPA_C10-12 CPA_C13-15   
BE  CPA_A01      499.889   23.790      NaN     1.171   3814.674     18.114   
    CPA_A02        4.976      NaN      NaN     0.038      0.408      0.346   
    CPA_A03        0.021      NaN      NaN       NaN      6.791      0.003   
    CPA_B          0.143    0.001      NaN     2.552      2.485      0.209   
    CPA_C10-12  1258.869    0.293      NaN     0.508   5278.772      5.560   
...                  ...      ...      ...       ...        ...        ...   
DOM D1           566.309   56.390   29.387   235.850   4804.886    876.118   
    D29X39      -421.926   -5.501      NaN     4.196    -98.855    -41.122   
    B2A3G       2661.075   66.295   21.143   121.088   3271.274    493.919   
    B1G         2805.458  117.184   50.530   361.134   7977.305   1328.915   
    P1          9625.119  427.959  129.963  1052.891  40035.175   4426.948   

                                                         ...     KR         \
                 CPA_C16   CPA_C17   CPA_C18    CPA_C19  ... P3_S13 P3_S15   
BE  CPA_A01        0.395     0.434     0.116      2.348  ...  0.029    NaN   
    CPA_A02      165.334    42.925     1.043      0.188  ...    NaN    NaN   
    CPA_A03        0.001     0.002     0.001      0.011  ...    NaN    NaN   
    CPA_B          0.122     0.444     0.021    323.535  ...    NaN    NaN   
    CPA_C10-12     1.631    19.403     1.909     39.087  ...  0.534  0.115   
...                  ...       ...       ...        ...  ...    ...    ...   
DOM D1           582.691   733.492   638.051    743.379  ...    NaN    NaN   
    D29X39       -48.407   -30.305   -31.531    -23.721  ...    NaN    NaN   
    B2A3G        348.356   419.927   383.109   1172.419  ...    NaN    NaN   
    B1G          882.640  1123.114   989.629   1892.077  ...    NaN    NaN   
    P1          3413.253  4300.258  2848.950  20583.584  ...    NaN    NaN   

                          WRL_REST                                      DOM  
                 P51G P5M   P3_S14  P3_S13 P3_S15    P51G    P5M         TU  
BE  CPA_A01     0.010 NaN   89.920   2.422  0.006  13.730  2.908   9625.119  
    CPA_A02       NaN NaN    1.112   0.084  0.009   0.002    NaN    427.959  
    CPA_A03       NaN NaN    1.133   0.002    NaN   0.005  0.002    129.963  
    CPA_B       0.004 NaN    0.305   0.013    NaN   0.090  0.024   1052.890  
    CPA_C10-12  0.001 NaN  962.896  41.137  0.878   1.144  0.020  40035.174  
...               ...  ..      ...     ...    ...     ...    ...        ...  
DOM D1            NaN NaN      NaN     NaN    NaN     NaN    NaN        NaN  
    D29X39        NaN NaN      NaN     NaN    NaN     NaN    NaN        NaN  
    B2A3G         NaN NaN      NaN     NaN    NaN     NaN    NaN        NaN  
    B1G           NaN NaN      NaN     NaN    NaN     NaN    NaN        NaN  
    P1            NaN NaN      NaN     NaN    NaN     NaN    NaN        NaN  

[2376 rows x 2597 columns]

Fixing some stuff, make it readable.

In [6]:
T.fillna(0, inplace=True)

In [7]:
T.index.names = ['region', 'product']
T.columns.names = ['region', 'product']

## Parse the full elements (Z, Y, x, V)

Quick check that it's the right size. The symmetric intermediate consumption table should only contain `CPA` codes.

In [8]:
Z_size = sum(v[:3] == 'CPA' for v in T.index.get_level_values(1))

We are guessing that FIGARO looks like the following:
- 27 EU regions with 64 sectors,
- 2 extra-EU regions with 64 sectors,
- 17 extra-EU regions with 30 sectors.

In [11]:
assert 27 * 64 +  2 * 64 + 17 * 30 == Z_size

In [12]:
Z = T.iloc[:Z_size,:Z_size]

In [14]:
Y_size = sum(v[:1] == 'P' for v in T.columns.get_level_values(1))

All regions seem to have the same final demand structure.

In [15]:
assert (27 + 2 + 17) * 5 == Y_size

In [16]:
Y = T.iloc[:Z_size, Z_size:Z_size+Y_size]

In [17]:
Y

region                      BE                                     BG         \
product                 P3_S14 P3_S13 P3_S15    P51G      P5M  P3_S14 P3_S13   
region   product                                                               
BE       CPA_A01      2185.504  0.000  0.000  28.199   55.233   1.605  0.000   
         CPA_A02        72.899  0.000  0.000   0.000  -15.524   0.001  0.000   
         CPA_A03        78.786  0.000  0.000   0.000    0.845   0.001  0.000   
         CPA_B           0.026  0.000  0.000   0.000   12.234   0.001  0.000   
         CPA_C10-12  15688.647  0.000  0.000   0.000  385.140  16.213  0.000   
...                        ...    ...    ...     ...      ...     ...    ...   
WRL_REST CPA_O84         0.000  0.000  0.000   0.000    0.000   0.000  0.000   
         CPA_P85         4.310  0.000  0.000   0.000    0.000   0.995  0.000   
         CPA_Q          14.386  0.000  0.000   0.000    0.000   1.487  0.000   
         CPA_R_S        55.100  8.904  0.531   0.378    0.000   5.644  0.403   
         CPA_T_U         0.000  0.000  0.000   0.000    0.000   0.000  0.000   

region                                    ...       KR                        \
product             P3_S15   P51G    P5M  ...   P3_S14  P3_S13 P3_S15   P51G   
region   product                          ...                                  
BE       CPA_A01     0.000  0.205  0.177  ...    0.570   0.029  0.000  0.010   
         CPA_A02     0.000  0.000  0.000  ...    0.000   0.000  0.000  0.000   
         CPA_A03     0.000  0.000  0.000  ...    0.002   0.000  0.000  0.000   
         CPA_B       0.000  0.000 -0.025  ...    0.002   0.000  0.000  0.004   
         CPA_C10-12  0.000  0.000  0.000  ...   27.404   0.534  0.115  0.001   
...                    ...    ...    ...  ...      ...     ...    ...    ...   
WRL_REST CPA_O84     0.000  0.000  0.000  ...    0.000   0.000  0.000  0.000   
         CPA_P85     0.000  0.000  0.000  ...  119.615   0.000  0.000  0.000   
         CPA_Q       0.000  0.000  0.000  ...   88.002   0.000  0.000  0.000   
         CPA_R_S     1.105  0.000  0.000  ...  318.176  16.606  1.453  2.477   
         CPA_T_U     0.000  0.000  0.000  ...    0.000   0.000  0.000  0.000   

region                      WRL_REST                                          
product              P5M      P3_S14      P3_S13     P3_S15      P51G    P5M  
region   product                                                              
BE       CPA_A01     0.0      89.920       2.422      0.006    13.730  2.908  
         CPA_A02     0.0       1.112       0.084      0.009     0.002  0.000  
         CPA_A03     0.0       1.133       0.002      0.000     0.005  0.002  
         CPA_B       0.0       0.305       0.013      0.000     0.090  0.024  
         CPA_C10-12  0.0     962.896      41.137      0.878     1.144  0.020  
...                  ...         ...         ...        ...       ...    ...  
WRL_REST CPA_O84     0.0   30087.713  587906.103    278.900  3637.520  0.197  
         CPA_P85     0.0  151660.571  210916.531   9532.537  1023.578 -0.385  
         CPA_Q       0.0  271388.263    1004.492  25890.458  1463.253 -0.117  
         CPA_R_S     0.0  124350.705   35562.795  22611.348  3370.817  1.854  
         CPA_T_U     0.0   51845.146       0.000      0.000     0.000  0.000  

[2366 rows x 230 columns]

In [18]:
x = T[('DOM','TU')].iloc[:Z_size]

In [19]:
x

region    product   
BE        CPA_A01         9625.119
          CPA_A02          427.959
          CPA_A03          129.963
          CPA_B           1052.890
          CPA_C10-12     40035.174
                           ...    
WRL_REST  CPA_O84       698087.625
          CPA_P85       451071.348
          CPA_Q         394633.302
          CPA_R_S       285552.044
          CPA_T_U        51845.146
Name: (DOM, TU), Length: 2366, dtype: float64

Relative error for the balance of outputs.

In [20]:
abs(((Z.sum(1) + Y.sum(1)) - x)/x).nlargest(20)

region  product   
LU      CPA_C19            inf
SI      CPA_C19       0.089623
LU      CPA_A03       0.029197
        CPA_C33       0.015174
MT      CPA_C19       0.006861
        CPA_C30       0.003571
LV      CPA_C19       0.002812
MT      CPA_C24       0.002404
        CPA_C16       0.001805
HR      CPA_C24       0.001086
CY      CPA_C26       0.000710
CZ      CPA_H50       0.000535
CY      CPA_M74_75    0.000491
        CPA_N77       0.000470
LU      CPA_C21       0.000454
CY      CPA_C19       0.000435
        CPA_M71       0.000415
LV      CPA_S95       0.000398
CY      CPA_C30       0.000359
LV      CPA_A03       0.000341
dtype: float64

Oh, hello Luxembourg.

In [21]:
V = T.iloc[Z_size:,:Z_size]

In [22]:
V

region                BE                                                    \
product          CPA_A01  CPA_A02  CPA_A03     CPA_B CPA_C10-12 CPA_C13-15   
region product                                                               
DOM    D21X31    370.872   20.795    7.020    24.971   1313.086    167.008   
       CIFFOB      0.000    0.000    0.000     0.000      0.000      0.000   
       OP_RES      0.000    0.000    0.000     0.000      0.000      0.000   
       OP_NRES     0.000    0.000    0.000     0.000      0.000      0.000   
       TUADJ    6819.661  310.775   79.433   691.757  32057.870   3098.033   
       D1        566.309   56.390   29.387   235.850   4804.886    876.118   
       D29X39   -421.926   -5.501    0.000     4.196    -98.855    -41.122   
       B2A3G    2661.075   66.295   21.143   121.088   3271.274    493.919   
       B1G      2805.458  117.184   50.530   361.134   7977.305   1328.915   
       P1       9625.119  427.959  129.963  1052.891  40035.175   4426.948   

region                                                   ...    WRL_REST  \
product          CPA_C16   CPA_C17   CPA_C18    CPA_C19  ...     CPA_J61   
region product                                           ...               
DOM    D21X31     58.751    80.222    51.548   1051.726  ...    2766.618   
       CIFFOB      0.000     0.000     0.000      0.000  ...       0.000   
       OP_RES      0.000     0.000     0.000      0.000  ...       0.000   
       OP_NRES     0.000     0.000     0.000      0.000  ...       0.000   
       TUADJ    2530.613  3177.144  1859.321  18691.507  ...  129620.711   
       D1        582.691   733.492   638.051    743.379  ...       0.000   
       D29X39    -48.407   -30.305   -31.531    -23.721  ...       0.000   
       B2A3G     348.356   419.927   383.109   1172.419  ...       0.000   
       B1G       882.640  1123.114   989.629   1892.077  ...  111933.578   
       P1       3413.253  4300.258  2848.950  20583.584  ...  241554.289   

region                                                                      \
product         CPA_J62_63       CPA_K     CPA_L68     CPA_M_N     CPA_O84   
region product                                                               
DOM    D21X31       71.314    1691.946    2155.471    4507.720    9557.765   
       CIFFOB        0.000       0.000       0.000       0.000       0.000   
       OP_RES        0.000       0.000       0.000       0.000       0.000   
       OP_NRES       0.000       0.000       0.000       0.000       0.000   
       TUADJ     45432.663  252238.386  150568.582  239870.602  313382.171   
       D1            0.000       0.000       0.000       0.000       0.000   
       D29X39        0.000       0.000       0.000       0.000       0.000   
       B2A3G         0.000       0.000       0.000       0.000       0.000   
       B1G       65843.024  331566.874  414409.654  331806.963  384705.454   
       P1       111275.687  583805.260  564978.236  571677.565  698087.625   

region                                                         
product            CPA_P85       CPA_Q     CPA_R_S    CPA_T_U  
region product                                                 
DOM    D21X31     4572.581    6325.123    4340.254      0.000  
       CIFFOB        0.000       0.000       0.000      0.000  
       OP_RES        0.000       0.000       0.000      0.000  
       OP_NRES       0.000       0.000       0.000      0.000  
       TUADJ    157445.619  178889.774  148595.851      0.000  
       D1            0.000       0.000       0.000      0.000  
       D29X39        0.000       0.000       0.000      0.000  
       B2A3G         0.000       0.000       0.000      0.000  
       B1G      293625.729  215743.528  136956.195  51845.146  
       P1       451071.348  394633.302  285552.046  51845.146  

[10 rows x 2366 columns]

In [23]:
V_Y = T.iloc[Z_size:,Z_size:Z_size+Y_size]

In [24]:
V_Y

region                 BE                                           BG  \
product            P3_S14   P3_S13 P3_S15      P51G      P5M    P3_S14   
region product                                                           
DOM    D21X31   12452.843  325.415  2.383  3160.199  219.995  2480.565   
       CIFFOB       0.000    0.000  0.000     0.000    0.000   484.448   
       OP_RES       0.000    0.000  0.000     0.000    0.000  1025.143   
       OP_NRES      0.000    0.000  0.000     0.000    0.000 -3494.971   
       TUADJ        0.000    0.000  0.000     0.000    0.000     0.000   
       D1           0.000    0.000  0.000     0.000    0.000     0.000   
       D29X39       0.000    0.000  0.000     0.000    0.000     0.000   
       B2A3G        0.000    0.000  0.000     0.000    0.000     0.000   
       B1G          0.000    0.000  0.000     0.000    0.000     0.000   
       P1           0.000    0.000  0.000     0.000    0.000     0.000   

region                                            ...     KR                \
product          P3_S13  P3_S15     P51G     P5M  ... P3_S14 P3_S13 P3_S15   
region product                                    ...                        
DOM    D21X31   136.986  13.075  184.684  -4.513  ...    0.0    0.0    0.0   
       CIFFOB    28.084   0.992  201.713  23.020  ...    0.0    0.0    0.0   
       OP_RES     0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       OP_NRES    0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       TUADJ      0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       D1         0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       D29X39     0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       B2A3G      0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       B1G        0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   
       P1         0.000   0.000    0.000   0.000  ...    0.0    0.0    0.0   

region                   WRL_REST                          
product        P51G  P5M   P3_S14 P3_S13 P3_S15 P51G  P5M  
region product                                             
DOM    D21X31   0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       CIFFOB   0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       OP_RES   0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       OP_NRES  0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       TUADJ    0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       D1       0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       D29X39   0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       B2A3G    0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       B1G      0.0  0.0      0.0    0.0    0.0  0.0  0.0  
       P1       0.0  0.0      0.0    0.0    0.0  0.0  0.0  

[10 rows x 230 columns]

In [25]:
V_x = T[('DOM','TU')].iloc[Z_size:]

## Derive coefficients

In [26]:
A = Z.div(x,axis=1)

In [27]:
A.sum().nlargest()

region  product
CY      CPA_H51    1.247094
NO      CPA_C19    1.050179
NL      CPA_C19    0.922257
EE      CPA_H51    0.918914
LT      CPA_C19    0.911330
dtype: float64

In [28]:
S = V/x

In [29]:
A.sum() + S.loc[('DOM','B1G')]

region    product   
BE        CPA_A01       0.961468
          CPA_A02       0.951273
          CPA_A03       0.945885
          CPA_B         0.976262
          CPA_C10-12    0.967203
                          ...   
WRL_REST  CPA_O84       0.986309
          CPA_P85       0.989863
          CPA_Q         0.983973
          CPA_R_S       0.984802
          CPA_T_U       1.000000
Length: 2366, dtype: float64

In [44]:
A[A.isna().sum()>0]

region              BE                                                      \
product        CPA_A01 CPA_A02 CPA_A03 CPA_B CPA_C10-12 CPA_C13-15 CPA_C16   
region product                                                               
BE     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
BG     CPA_T       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
       CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
CZ     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
DK     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
DE     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
EE     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
IE     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
EL     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
ES     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
FR     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
HR     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
IT     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
CY     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
LV     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
LT     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
LU     CPA_C19     0.0     0.0     0.0   0.0        0.0        0.0     0.0   
       CPA_E36     0.0     0.0     0.0   0.0        0.0        0.0     0.0   
       CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
HU     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
MT     CPA_A02     0.0     0.0     0.0   0.0        0.0        0.0     0.0   
       CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
NL     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
AT     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
PL     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
PT     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
RO     CPA_T       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
       CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
SI     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
SK     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
FI     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
SE     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
UK     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
US     CPA_U       0.0     0.0     0.0   0.0        0.0        0.0     0.0   
AU     CPA_T_U     0.0     0.0     0.0   0.0        0.0        0.0     0.0   
CN     CPA_T_U     0.0     0.0     0.0   0.0        0.0        0.0     0.0   
JP     CPA_T_U     0.0     0.0     0.0   0.0        0.0        0.0     0.0   

region                                  ... WRL_REST                           \
product        CPA_C17 CPA_C18 CPA_C19  ...  CPA_J61 CPA_J62_63 CPA_K CPA_L68   
region product                          ...                                     
BE     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
BG     CPA_T       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
       CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
CZ     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
DK     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
DE     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
EE     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0     0.0   
IE     CPA_U       0.0     0.0     0.0  ...      0.0        0.0   0.0

In [45]:
A.fillna(0, inplace=True)

In [46]:
I = csc_matrix(np.eye(A.shape[0]))

In [47]:
A_csc = csc_matrix(A)

In [48]:
L = spsolve(I-A_csc, I.todense())

In [49]:
L

array([[1.07234947e+00, 6.05076064e-02, 9.06354885e-04, ...,
        2.21783878e-05, 1.92194771e-05, 0.00000000e+00],
       [7.67622922e-04, 1.00012588e+00, 8.94536344e-05, ...,
        1.58221969e-06, 1.62140718e-06, 0.00000000e+00],
       [3.90617935e-05, 8.58097985e-06, 1.00000697e+00, ...,
        1.31800756e-07, 1.31635038e-07, 0.00000000e+00],
       ...,
       [2.42176770e-04, 2.60163288e-04, 1.90598769e-04, ...,
        1.11319092e+00, 2.33437223e-02, 0.00000000e+00],
       [3.42640250e-04, 3.92152289e-04, 3.19985209e-04, ...,
        1.58353015e-02, 1.08428844e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [50]:
L_df = pd.DataFrame(L,
                   index=A.index,
                   columns=A.columns)

In [51]:
L_df

region                     BE                                           \
product               CPA_A01   CPA_A02   CPA_A03     CPA_B CPA_C10-12   
region   product                                                         
BE       CPA_A01     1.072349  0.060508  0.000906  0.002164   0.118971   
         CPA_A02     0.000768  1.000126  0.000089  0.000371   0.000429   
         CPA_A03     0.000039  0.000009  1.000007  0.000010   0.000210   
         CPA_B       0.000690  0.002352  0.002369  1.003307   0.000482   
         CPA_C10-12  0.164434  0.012183  0.002345  0.003329   1.172892   
...                       ...       ...       ...       ...        ...   
WRL_REST CPA_O84     0.000312  0.000361  0.000292  0.000341   0.000386   
         CPA_P85     0.000245  0.000321  0.000256  0.000315   0.000290   
         CPA_Q       0.000242  0.000260  0.000191  0.000211   0.000301   
         CPA_R_S     0.000343  0.000392  0.000320  0.000366   0.000430   
         CPA_T_U     0.000000  0.000000  0.000000  0.000000   0.000000   

region                                                                  ...  \
product             CPA_C13-15   CPA_C16   CPA_C17   CPA_C18   CPA_C19  ...   
region   product                                                        ...   
BE       CPA_A01      0.006131  0.004509  0.002544  0.001309  0.001316  ...   
         CPA_A02      0.000290  0.054808  0.011218  0.001768  0.000113  ...   
         CPA_A03      0.000013  0.000010  0.000010  0.000010  0.000008  ...   
         CPA_B        0.000650  0.000670  0.000774  0.000451  0.018443  ...   
         CPA_C10-12   0.005897  0.004189  0.009076  0.004226  0.004846  ...   
...                        ...       ...       ...       ...       ...  ...   
WRL_REST CPA_O84      0.000292  0.000348  0.000335  0.000299  0.000514  ...   
         CPA_P85      0.000230  0.000283  0.000277  0.000249  0.000506  ...   
         CPA_Q        0.000194  0.000262  0.000245  0.000226  0.000257  ...   
         CPA_R_S      0.000306  0.000374  0.000354  0.000326  0.000550  ...   
         CPA_T_U      0.000000  0.000000  0.000000  0.000000  0.000000  ...   

region                   WRL_REST                                            \
product                   CPA_J61    CPA_J62_63         CPA_K       CPA_L68   
region   product                                                              
BE       CPA_A01     8.304805e-06  1.127076e-05  8.918587e-06  5.300649e-06   
         CPA_A02     1.148397e-06  1.314535e-06  1.081493e-06  8.622828e-07   
         CPA_A03     6.248147e-08  9.967581e-08  1.083948e-07  4.146790e-08   
         CPA_B       5.828747e-06  5.468290e-06  4.215207e-06  5.239632e-06   
         CPA_C10-12  2.774615e-05  3.907830e-05  3.356337e-05  1.910231e-05   
...                           ...           ...           ...           ...   
WRL_REST CPA_O84     1.310215e-02  8.031168e-03  9.603423e-03  9.015941e-03   
         CPA_P85     1.001158e-02  1.207975e-02  1.275678e-02  2.831912e-03   
         CPA_Q       7.435961e-03  7.364392e-03  1.414931e-02  3.063443e-03   
         CPA_R_S     1.558385e-02  1.475897e-02  1.116688e-02  3.821907e-03   
         CPA_T_U     0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

region                                                                       \
product                   CPA_M_N       CPA_O84       CPA_P85         CPA_Q   
region   product                                                              
BE       CPA_A01     1.230709e-05  1.645889e-05  1.344624e-05  2.217839e-05   
         CPA_A02     1.293931e-06  1.411856e-06  1.241408e-06  1.582220e-06   
         CPA_A03     8.694116e-08  9.927913e-08  7.515174e-08  1.318008e-07   
         CPA_B       6.763872e-06  8.295079e-06  5.240879e-06  7.876060e-06   
         CPA_C10-12  4.534612e-05  7.409839e-05  6.587532e-05  8.959677e-05   
...                           ...           ...           ...           ...   
WRL_REST CPA_O84     1.247593e-02  1.0